# The Jam Machine 

## decription of the model here
### - Sequence length 1024
### - whole dataset
### - tristan's encoder
### - improved features
    - pushing on Hugging face
    - writing the tokenized dataset on disk
    - loading the tokenized dataset --> saves times for big datasets


# Imports

In [ ]:
## THIS NEEDS TO BE CLEANED - NOT EVERYTHING IS NECESSARY
# basic packages requirements
!pip install wandb
import os
from pathlib import Path
import glob
import random
import shutil
import tqdm
import matplotlib.pyplot as plt
import typing
import json
# tensorflow
# import tensorflow as tf
# from tensorflow.keras import preprocessing
# from tensorflow.keras import models, layers
# tf.config.list_physical_devices("GPU")
# transformers
!pip install transformers tokenizers
from transformers import PreTrainedTokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import GPT2Config, GPT2LMHeadModel
from transformers import TrainingArguments, Trainer
# tokenizer
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
# torch
import torch

# install requirements
# !pipenv lock -r > requirements.txt
# !pip install -r requirements.txt

# Setup requirements
Here you should set the path for saving your trained model etc

In [ ]:
!git clone https://github.com/m41w4r3exe/the-jam-machine.git

# move into the git folder to have all scripts in the local folder
os.chdir("./the-jam-machine")

# check that we re in the right happy place
print(f"Current working directory: {os.getcwd()}")
print(f"Files and Folders in directory:")
os.listdir()

# mount google drive
from google.colab import drive
drive.mount('/content/drive')

# define path
drive_path = Path("/content/drive/MyDrive/the_jam_machine/")
# midi_path = Path(f"{drive_path}/midi_data")
# encoded_midi_path = Path(f"{drive_path}/midi_encoded")

# DATA
encoded_midi_path = Path(f"{drive_path}/midi_encoded_by_tristan")

# TOKENIZER
tokenizer_path = Path(f"{drive_path}/tokenizer")
tokenizer_file = "tokenizer.json"
load_tokenizer_pretrained = True
load_tokenized_datasets = True

# MODEL
model_path = Path(f"{drive_path}/model_jean_1024_8layers")
if not os.path.exists(model_path):
  print(f"creating {model_path}")
  os.mkdir(model_path)

# HuggingFace repo
hf_repo = "misnaej/the-jam-machine-1024"
hf_token = 

# device = "cpu"
device = "cuda" 
train_from_scratch = False

if train_from_scratch:
  train_from_this_checkpoint = None

elif not train_from_scratch:
  train_from_this_checkpoint = "checkpoint-86906"
  additionnal_epochs_to_run = 0
  checkpoint_path = Path(f"{model_path}/{train_from_this_checkpoint}")


# Encoding Midi into Text (Done already)
from `mymusic.mid` files from `midi_path` to `music.txt` files in `encoded_midi_path`

In [ ]:
#@title File encoding
#!python encoder_mlike.py --midi_path {midi_path} --encoded_midi_path {encoded_midi_path}

# Dataset
- load the encoded data
- training set - ?? % (done already by Tristan's encoding)
- validation set - ?? % (done already by Tristan's encoding)

- Tokenizing


## Loading the data

Define data path and define individual path for every files

In [ ]:
# write full filepath to list
def full_filepath_to_list(dataset_folder, file_limit=1000):
  dataset_files = os.listdir(dataset_folder)
  dataset_files = [f"{dataset_folder}/{file}" for count, file in enumerate(dataset_files) if count < file_limit]
  print(dataset_files)
  return dataset_files

if not load_tokenized_datasets:
  dataset_path = encoded_midi_path
  dataset_path_all = os.path.join(dataset_path, "all")
  dataset_path_train = os.path.join(dataset_path, "train")
  dataset_path_valid = os.path.join(dataset_path, "valid")
  files_train = full_filepath_to_list(dataset_path_train) # training set
  files_valid = full_filepath_to_list(dataset_path_valid) # validation set

## Put data into dictionnary

In [ ]:
# TO CHANGE  - this will probably give us some memory issues 
# Dictionnary stage to skip here and load the date into the tokenizer directly from the text files
# show the first 2 items
def show_n_first_dataset_entries(dataset, n=2):
  for idx, key in enumerate(dataset.values()):
    if idx >= n:
      break
    else:
      print(key)

def dataset_in_dictionary(files):
  dataset_dict = {}
  dataset_size = 0
  for _ , file in enumerate(files):
    line_count = 0

    filename = file.split("/")[-1].split(".")[0]
    file_content = open(file, "r")

    while True:
      line_count += 1
      line =  file_content.readline().rstrip("\n") 
      dataset_size += 1
      if line_count % 10000 == 0:
        print(f"Writting - {filename} - count={line_count}")
        print(line)  

      if not line: # break when file is over
        print(f"Breaking - {filename} - count={line_count}")
        print(line)
        
        break

      dataset_dict[f"{filename}_{line_count}"] = line

    

  show_n_first_dataset_entries(dataset_dict, n=2)

  return dataset_dict, dataset_size

if not load_tokenized_datasets:
  print("=============")  
  print("Training data")
  print("=============")
  dataset_train_dict, dataset_train_size = dataset_in_dictionary(files_train)
  print(f"dataset_train_size = {dataset_train_size}")
  print("===============")
  print("Validation data")
  print("===============")
  dataset_valid_dict, dataset_valid_size = dataset_in_dictionary(files_valid)
  print(f"dataset_valid_size = {dataset_valid_size}")
  # print(f"dataset length: {len(dataset_train_dict.items()}")

# Tokenizer



## Train the TOKENIZER

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers.trainers import WordLevelTrainer
from transformers import PreTrainedTokenizerFast


# making an iterator to go through the dictionary 
# CHANG THIS TO LOAD DIRECTLY FROM THE FILES TO SAVE MEMORY
def get_training_corpus():
    for song in dataset_train_dict.values():
        yield song

def define_and_save_tokenizer(
  saved_tokenizer="tokenizer.json", 
  tokenizer_path="./"
):
  # using a Word Level Tokemizer
  tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
  # separating vocabulary on whitespaces (pretokenizer)
  tokenizer.pre_tokenizer = WhitespaceSplit()
  # set up the trainer 
  trainer = WordLevelTrainer(
      special_tokens=["[UNK]", "[PAD]", "[MASK]"]
  )
  # special tokens I took the ones tristan used but I am not sure I understand all of them 
  # [UNK] always need and UNK token
  # [PAD] if we need to do some pading
  # [MASK] if when we need to mask, in order to predict the future without seeing it
  # [CLS] what is it ? it is for BERT - the last token of the sequence
  # [SEP] what is it ? - SEP can be used with very short sequences

  # get corpus
  training_corpus = get_training_corpus()
  # train tokenizer
  tokenizer.train_from_iterator(training_corpus, trainer=trainer)
  # save tokenizer
  tokenizer.save(f"{tokenizer_path}/{saved_tokenizer}")

  return tokenizer

if not load_tokenizer_pretrained:
  tokenizer = define_and_save_tokenizer(tokenizer_path=tokenizer_path)


## Or load the trained tokenizer


In [ ]:
if load_tokenizer_pretrained:
  from transformers import PreTrainedTokenizerFast
  # if device == "cuda":
  #   tokenizer = PreTrainedTokenizerFast(tokenizer_file=f"{tokenizer_path}/{tokenizer_file}")
  # else:
    # tokenizer = PreTrainedTokenizerFast(tokenizer_file=f"{tokenizer_path}/{tokenizer_file}")
  tokenizer = PreTrainedTokenizerFast(tokenizer_file=f"{tokenizer_path}/{tokenizer_file}")

 ## Make it ready for GPT2 : add the pad_token

In [ ]:
def format_tokenizer_for_transformers_classes(saved_tokenizer="tokenizer.json"):
  # PreTrainedTokenizerFast makes the tokenizer usable by the transformer
  tokenizer = PreTrainedTokenizerFast(tokenizer_file = saved_tokenizer)
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})

  return tokenizer

tokenizer = format_tokenizer_for_transformers_classes(
    saved_tokenizer=f"{tokenizer_path}/{tokenizer_file}"
)

## Visualize the Vocabulary

In [ ]:
print(tokenizer.vocab_size)
tokenizer.get_vocab()

## Tokenize the DATASET and prepare it for the model

In [ ]:
def tokenize_function(data_to_tokenize):
  # this make the tokenized data ready for the model that requires a
    tokenized_data = tokenizer(
        data_to_tokenize,
        truncation=True,
        padding=True,
        max_length=1024,
    )
    return {
        "input_ids": tokenized_data["input_ids"]
    } 

def tokenize_dataset(dataset_dictionary, tokenizer, limit=None):
  if limit is None:
    limit = len(dataset_dictionary.items())

  tokenized_dictionary=[]
  for idx, file in enumerate(dataset_dictionary.values()):
    # print(file)
    if idx<limit:
      tokenized_dictionary.append(tokenize_function(file))
      # print(encoded_file)

  return tokenized_dictionary

def write_to_json(file, path):
  with open(f"{path}", 'w') as json_file:
    json.dump(file, json_file)

def load_json(path):
  with open(f"{path}", 'r') as json_file:
    file = json.load(json_file)
    return file

if not load_tokenized_datasets:
  dataset_train_tokenized = tokenize_dataset(dataset_train_dict,tokenizer)
  dataset_val_tokenized = tokenize_dataset(dataset_valid_dict,tokenizer)

  write_to_json(dataset_val_tokenized, f"{model_path}/dataset_val_tokenized.json")
  write_to_json(dataset_train_tokenized, f"{model_path}/dataset_train_tokenized.json")

In [ ]:
if load_tokenized_datasets:
  dataset_train_tokenized = load_json(f"{model_path}/dataset_train_tokenized.json")
  dataset_val_tokenized = load_json(f"{model_path}/dataset_val_tokenized.json")

In [ ]:
# checking that the data is in the correct shape to be input to the model
assert list(dataset_train_tokenized[0]) == ["input_ids"], list(dataset_train_tokenized[0])
print(type(dataset_train_tokenized[0]))
# Check a few samples
for i, ii in enumerate(dataset_train_tokenized):
  if i > 4: # print only the first 5
    break
  else: 
    print("----")
    # print(ii)
    print(dataset_train_tokenized[i])

# Define the model

## Data collator.
[Huggingface](https://huggingface.co/transformers/v4.8.1/main_classes/data_collator.html)

"
Data collators are objects that will form a batch by using a list of dataset elements as input. These elements are of the same type as the elements of train_dataset or eval_dataset.

To be able to build batches, data collators may apply some processing (like padding). Some of them (like DataCollatorForLanguageModeling) also apply some random data augmentation (like random masking) oin the formed batch.
"

"
**class transformers.data.data_collator.DataCollatorForLanguageModeling**
- tokenizer (PreTrainedTokenizer or PreTrainedTokenizerFast) – The tokenizer used for encoding the data.
- mlm (bool, optional, defaults to True) – Whether or not to use masked language modeling. If set to False, the labels are the same as the inputs with the padding tokens ignored (by setting them to -100). Otherwise, the labels are -100 for non-masked tokens and the value to predict for the masked token"

## Predicting the next note using GPT2
[HuggingFace](https://huggingface.co/transformers/v4.8.1/task_summary.html#)

[Class GPT2Config](https://huggingface.co/transformers/v4.8.1/_modules/transformers/configuration_gpt2.html)
This is the config that needs to be passed to GPT2Model Classes.

[Class GPT2LMHeadModel](https://huggingface.co/docs/transformers/model_doc/gpt2#transformers.GPT2LMHeadModel)
Here we use the GPT2 Model transformer with a language modeling head on top (linear layer with weights tied to the input embeddings).



In [ ]:
# data collator
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# model
from transformers import GPT2Config, GPT2LMHeadModel
model_config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    pad_token_id=tokenizer.pad_token_id,
    n_embd=512, #Dimensionality of the embeddings and hidden states
    n_head=8,#Number of attention heads for each attention layer in the Transformer encoder
    n_layer=8, #Number of hidden layers in the Transformer encoder
    n_positions=1024, # maximum sequence length default = 1024
)
model = GPT2LMHeadModel(model_config)
model

# Testing the-jam-machine
Tristan's script

In [ ]:
import random
import matplotlib.pyplot as plt
import torch

inputs = [random.choice(dataset_train_tokenized)]
inputs = data_collator(inputs)
assert list(inputs.keys()) == ["input_ids", "attention_mask", "labels"], list(inputs.keys())
print("input_ids:", inputs["input_ids"])
print("")

outputs = model(**inputs)
assert list(outputs.keys()) == ["loss", "logits", "past_key_values"], list(outputs.keys())
print("logits:", outputs["logits"])

plt.plot(outputs["logits"].detach().numpy()[0][0])
plt.title("Logits")
plt.show()
plt.close()

activations = torch.nn.functional.softmax(outputs["logits"], dim=-1)
plt.plot(activations.detach().numpy()[0][0])
plt.title("Activations")
plt.show()
plt.close()

# Train the-jam-machine

## Initialize Weight and Biases

In [ ]:
# Weight and Biases
import wandb
os.environ["WANDB_NOTEBOOK_NAME"] = "the-jam-machine_2022_12_04"
wandb.init(project="the-jam-machine-test")

## Training

Check those links (Hugging Face): 
- [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer)
- [trainer_utils.py](https://github.dev/huggingface/transformers/blob/main/src/transformers/trainer_utils.py)

- [TrainingArguments (training_args.py)](https://github.com/huggingface/transformers/blob/main/src/transformers/training_args.py)

In [ ]:
checkpoint_path

In [ ]:
# defining training params
prop_dataset_to_pass_before_evaluating = 1/10
data_samples_to_pass_before_evaluating = prop_dataset_to_pass_before_evaluating*len(dataset_train_tokenized)
per_device_train_batch_size = 8
device_nb = 1
num_train_epochs = 2
eval_steps = round(data_samples_to_pass_before_evaluating / num_train_epochs / per_device_train_batch_size / device_nb)

In [ ]:
from transformers import TrainingArguments, Trainer
import os
# Create the trainer.
print("Creating trainer...")
training_args = TrainingArguments(
    output_dir=model_path, # where model and checkpoints will be saved
    overwrite_output_dir=True,# self-explanatory
    num_train_epochs=num_train_epochs, # defaults to 3.0
    evaluation_strategy="steps", # Evaluation is done (and logged) every eval_steps (so not at the end of every epoch)
    eval_steps=eval_steps, # Number of update steps (backprob) between two evaluations
    learning_rate=5e-4, #initial learning rate for :class:`~transformers.AdamW` optimizer.
    weight_decay=0.15, # Default o.
    warmup_steps=5_000,
    lr_scheduler_type="cosine", # Default linear
    fp16=False, # Default False. Is True in HF.
    bf16_full_eval=False,
    per_device_train_batch_size=per_device_train_batch_size, #default value is 8
    seed=42, # default seed=42
    save_strategy="steps", # 
    save_steps=eval_steps*2,
    save_total_limit=5,
    logging_strategy="steps",
    logging_first_step=True,
    logging_steps=eval_steps,
    logging_dir=os.path.join(model_path, "logs"),
    prediction_loss_only=False,
    report_to="wandb",
    load_best_model_at_end=True,
    metric_for_best_model="loss", # "eval_loss"
    push_to_hub = True,
    hub_strategy="end", #huggingface repo -> saves the final model at the end of training
    hub_model_id=hf_repo,
    hub_token=hf_token,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset_train_tokenized,
    eval_dataset=dataset_val_tokenized,
)

# Save the tokenizer.
tokenizer.save_pretrained(model_path)

# Train the model.
if train_from_scratch:
  trainer.train()
else:
  if train_from_this_checkpoint is not None:
    trainer.args.num_train_epochs += additionnal_epochs_to_run
    trainer.resume_from_checkpoint = checkpoint_path # that is not used by the trainer but potentially for the metrics # to check
    trainer.train(checkpoint_path)

  else:
    pass
# Save the model.
model.save_pretrained(model_path)
# Push to Hugging Face
trainer.push_to_hub(commit_message = "End of training")